In [1]:
# Lib Imports
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, LinearRegression, LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [2]:
# Setting Pandas column display option
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
idee = pd.read_csv('train_identity.csv')
transaction = pd.read_csv('train_transaction.csv')

In [4]:
def pipeline(idee, transaction):
    merge = transaction.merge(idee, how='outer', on='TransactionID')
    objects = merge.select_dtypes('object')
    objects.fillna("Unknown", inplace=True)
    objects = objects.astype('category')
    print(objects.info())
    return objects
objects = pipeline(idee,transaction)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   ProductCD      590540 non-null  category
 1   card4          590540 non-null  category
 2   card6          590540 non-null  category
 3   P_emaildomain  590540 non-null  category
 4   R_emaildomain  590540 non-null  category
 5   M1             590540 non-null  category
 6   M2             590540 non-null  category
 7   M3             590540 non-null  category
 8   M4             590540 non-null  category
 9   M5             590540 non-null  category
 10  M6             590540 non-null  category
 11  M7             590540 non-null  category
 12  M8             590540 non-null  category
 13  M9             590540 non-null  category
 14  id_12          590540 non-null  category
 15  id_15          590540 non-null  category
 16  id_16          590540 non-null  category
 17  id_23     

In [5]:
value_counts_list = []
obj_columns_list = objects.columns.to_list()
for column in obj_columns_list:
    df = objects[column].value_counts().reset_index()
    value_counts_list.append(df)

objects_value_counts_df = pd.concat(value_counts_list, axis=1)
objects_value_counts_df.tail(1700)

,index,ProductCD,index,card4,index,card6,index,P_emaildomain,index,R_emaildomain,index,M1,index,M2,index,M3,index,M4,index,M5,index,M6,index,M7,index,M8,index,M9,index,id_12,index,id_15,index,id_16,index,id_23,index,id_27,index,id_28,index,id_29,index,id_30,index,id_31,index,id_33,index,id_34,index,id_35,index,id_36,index,id_37,index,id_38,index,DeviceType,index,DeviceInfo
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chrome 51.0 for android,45.0,1024x552,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moto,77
88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,safari 9.0,43.0,1920x1081,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blade V6 Plus Build/MRA58K,76
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,samsung browser 5.4,43.0,1440x810,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SM-G930T Build/NRD90M,76
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,chrome 53.0 for android,42.0,1679x1049,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LG-K530 Build/MMB29M,75
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,samsung browser 5.2,41.0,2000x1125,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SM-G950U Build/R16NW,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QwestIE8,1
1783,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R8106,1
1784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0PJA2,1
1785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RCT6K03W13,1


In [6]:
objects['isFraud'] = transaction['isFraud']
device = objects[['DeviceInfo', 'isFraud']]
device

,DeviceInfo,isFraud
0,Unknown,0
1,Unknown,0
2,Unknown,0
3,Unknown,0
4,SAMSUNG SM-G892A Build/NRD90M,0
...,...,...
590535,Unknown,0
590536,Unknown,0
590537,Unknown,0
590538,Unknown,0


In [7]:
device = pd.get_dummies(device)

In [8]:
device

,isFraud,DeviceInfo_0PAJ5,DeviceInfo_0PJA2,DeviceInfo_0PM92,DeviceInfo_1016S,DeviceInfo_2PQ93,DeviceInfo_2PS64 Build/NRD90M,DeviceInfo_2PYB2,DeviceInfo_2PZC5,DeviceInfo_4003A,DeviceInfo_4009F,DeviceInfo_4013M Build/KOT49H,DeviceInfo_4027A Build/KOT49H,DeviceInfo_4034E,DeviceInfo_4034G,DeviceInfo_4047A Build/NRD90M,DeviceInfo_4047G Build/NRD90M,DeviceInfo_47418,DeviceInfo_5010G Build/MRA58K,DeviceInfo_5010S Build/MRA58K,DeviceInfo_5011A Build/NRD90M,DeviceInfo_5012G Build/MRA58K,DeviceInfo_5015A Build/LMY47I,DeviceInfo_5017A,DeviceInfo_5025G Build/LMY47I,DeviceInfo_5042A,DeviceInfo_5044A,DeviceInfo_5045I,DeviceInfo_5049W Build/NRD90M,DeviceInfo_5051A Build/MMB29M,DeviceInfo_5054N,DeviceInfo_5054S Build/LMY47V,DeviceInfo_5056A Build/MMB29M,DeviceInfo_5056N,DeviceInfo_5057M,DeviceInfo_5080A Build/MRA58K,DeviceInfo_5085B Build/MRA58K,DeviceInfo_5095I Build/MRA58K,DeviceInfo_6037B,DeviceInfo_6039A Build/LRX22G,DeviceInfo_6045I Build/LRX22G,DeviceInfo_6055B,DeviceInfo_7048A Build/LRX22G,DeviceInfo_7055A Build/KVT49L,DeviceInfo_7_Plus,DeviceInfo_8050G Build/LMY47I,DeviceInfo_8062 Build/MRA58K,DeviceInfo_8080 Build/LRX21M,DeviceInfo_831C,DeviceInfo_9002A,DeviceInfo_9003A Build/MRA58K,DeviceInfo_9008A Build/NRD90M,DeviceInfo_9022X,DeviceInfo_916,DeviceInfo_9203A Build/MRA58K,DeviceInfo_A0001,DeviceInfo_A0001 Build/MHC19Q,DeviceInfo_A1-850,DeviceInfo_A1601 Build/LMY47I,DeviceInfo_A3-A20,DeviceInfo_A37f,DeviceInfo_A3_mini,DeviceInfo_A463BG,DeviceInfo_A466BG,DeviceInfo_A5002,DeviceInfo_A50C+,DeviceInfo_A574BL Build/NMF26F,DeviceInfo_A577VL,DeviceInfo_A621R,DeviceInfo_A9,DeviceInfo_A952,DeviceInfo_A96 Build/LMY47I,DeviceInfo_A97,DeviceInfo_AERIAL,DeviceInfo_AERIAL Build/NRD90M,DeviceInfo_AKUS,DeviceInfo_ALCATEL,DeviceInfo_ALCATEL ONE TOUCH 5036A Build/JDQ39,DeviceInfo_ALCATEL ONE TOUCH 7040A Build/JDQ39,DeviceInfo_ALCATEL ONE TOUCH 7042A Build/JDQ39,DeviceInfo_ALCATEL ONE TOUCH 7047A Build/JDQ39,DeviceInfo_ALE-L21 Build/HuaweiALE-L21,DeviceInfo_ALE-L23 Build/HuaweiALE-L23,DeviceInfo_ALP-L09 Build/HUAWEIALP-L09,DeviceInfo_ALP-L09 Build/HUAWEIALP-L09S,DeviceInfo_ALTER,DeviceInfo_ALUMINI,DeviceInfo_AM508,DeviceInfo_ANE-LX3 Build/HUAWEIANE-LX3,DeviceInfo_AOC_U706G,DeviceInfo_ASTRO,DeviceInfo_ASUS,DeviceInfo_ASUS_A001,DeviceInfo_ASUS_P00J,DeviceInfo_ASUS_X008D Build/NRD90M,DeviceInfo_ASUS_X008DC Build/NRD90M,DeviceInfo_ASUS_X00DD,DeviceInfo_ASUS_X00DDA,DeviceInfo_ASUS_X00HD Build/NMF26F,DeviceInfo_ASUS_X00ID,DeviceInfo_ASUS_X015D Build/NRD90M,DeviceInfo_ASUS_X018D,DeviceInfo_ASUS_Z00AD Build/LRX21V,DeviceInfo_ASUS_Z00ED,DeviceInfo_ASUS_Z00UD,DeviceInfo_ASUS_Z017D,DeviceInfo_ASUS_Z017DA,DeviceInfo_ASUS_Z01BDA,DeviceInfo_ASUS_Z01BDC,DeviceInfo_ASUS_Z01KD,DeviceInfo_ATT,DeviceInfo_ATT-IE11,DeviceInfo_AX1060,DeviceInfo_AX1070,DeviceInfo_AX705,DeviceInfo_AX820 Build/MRA58K,DeviceInfo_AX821 Build/MRA58K,DeviceInfo_AX920,DeviceInfo_AX921 Build/MRA58K,DeviceInfo_Advance,DeviceInfo_Alcatel,DeviceInfo_Alcatel_4060A,DeviceInfo_Alcatel_4060O Build/MMB29M,DeviceInfo_Alcatel_5044R Build/NRD90M,DeviceInfo_Alcatel_5054O,DeviceInfo_Alcatel_5056O,DeviceInfo_Alcatel_5098O Build/MMB29M,DeviceInfo_Alumini3 Build/MRA58K,DeviceInfo_Alumini3Plus,DeviceInfo_Android,DeviceInfo_Android 4.4.2,DeviceInfo_Android 5.1,DeviceInfo_Android 5.1.1,DeviceInfo_Android 6.0,DeviceInfo_Android 6.0.1,DeviceInfo_Android 7.0,DeviceInfo_Android 7.1.2,DeviceInfo_Aquaris,DeviceInfo_Aquaris U Plus Build/NMF26F,DeviceInfo_Aquaris V Build/N2G47H,DeviceInfo_Aquaris X Build/NMF26F,DeviceInfo_Aquaris X5 Plus Build/NMF26F,DeviceInfo_Aquaris_A4.5,DeviceInfo_Archos,DeviceInfo_Azumi_KINZO_A5_QL,DeviceInfo_B1-750,DeviceInfo_B1-790,DeviceInfo_B1-810,DeviceInfo_B3-A20,DeviceInfo_B3-A40,DeviceInfo_BAC-L03 Build/HUAWEIBAC-L03,DeviceInfo_BAH-L09 Build/HUAWEIBAH-L09,DeviceInfo_BBA100-1,DeviceInfo_BBA100-2,DeviceInfo_BBB100-1,DeviceInfo_BBB100-2,DeviceInfo_BBB100-3,DeviceInfo_BG2-W09,DeviceInfo_BLA-L09 Build/HUAWEIBLA-L09,DeviceInfo_BLA-L29 Build/HUAWEIBLA-L29,DeviceInfo_BLADE A520 Build/NRD90M,DeviceInfo_BL

In [13]:
X = device.drop(columns='isFraud')
y = device['isFraud']

under = RandomUnderSampler() 
over = RandomOverSampler()
sm = SMOTE()
ad = ADASYN()
lasso = LassoCV(tol=0.01, n_jobs=-1)

# X_rus, y_rus = under.fit_resample(X, y)
# X_ros, y_ros = over.fit_resample(X, y)
# X_sm, y_sm = sm.fit_resample(X, y)
X_ad, y_ad = ad.fit_resample(X, y)
# X_cs, y_cs = under.fit_resample(X_ros, y_ros)

def lasso_fit(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)
    fit = lasso.fit(X_train, y_train)
    penalized_features = pd.Series(fit.coef_, index=X_train.columns)
    imp_coef = pd.concat([penalized_features.sort_values().head(10),penalized_features.sort_values().tail(10)])
    return imp_coef

In [ ]:
lasso_fit(X_ad, y_ad)